In [2]:
# -*- coding: utf-8 -*-
# @Time    : 2019/05
# @Author  : zhangle
# @function   : data mining
# @Software: Jupyter

import pandas as pd
import multiprocessing
import numpy as np
import matplotlib.pyplot as plt
import xgboost as xgb
import lightgbm as lgb
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score

data= pd.read_csv('C://Users//zhangle//Desktop//data_set//train_1.csv')
train = data.iloc[:543990,:]
test = data.iloc[-200000:,:]
print(train.shape)
print(test.shape)

(543990, 27)
(200000, 27)


D:\anocanda\lib\site-packages\IPython\core\interactiveshell.py:2785: DtypeWarning: Columns (4,5,20,21) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
#数据预处理
train = train[train.gender != '\\N']
test = test[test.gender != '\\N']
train['gender'] = train['gender'].apply(lambda x : int(x))
test['gender'] = test['gender'].apply(lambda x : int(x))

train = train[train.age != '\\N']
test = test[test.age != '\\N']
train['age'] = train['age'].apply(lambda x : int(x))
test['age'] = test['age'].apply(lambda x : int(x))

train = train[train['2_total_fee'] != '\\N']
test = test[test['2_total_fee'] != '\\N']
test.loc[test['2_total_fee'] == '\\N','2_total_fee'] = 0.0
train['2_total_fee'] = train['2_total_fee'].apply(lambda x : float(x))
test['2_total_fee'] = test['2_total_fee'].apply(lambda x : float(x))

train = train[train['3_total_fee'] != '\\N']
test = test[test['3_total_fee'] != '\\N']
test.loc[test['3_total_fee'] == '\\N','3_total_fee'] = 0.0
train['3_total_fee'] = train['3_total_fee'].apply(lambda x : float(x))
test['3_total_fee'] = test['3_total_fee'].apply(lambda x : float(x))
data = pd.concat([train, test]).reset_index(drop=True).fillna(0) #合并之后重新建立索引 将空缺的地方补为0
data.head()

,service_type,is_mix_service,online_time,1_total_fee,2_total_fee,3_total_fee,4_total_fee,month_traffic,many_over_bill,contract_type,...,local_caller_time,service1_caller_time,service2_caller_time,gender,age,complaint_level,former_complaint_num,former_complaint_fee,current_service,user_id
0,4,0,85,295.96,296.20,296.00,296.80,3813.614698,0,1,...,108.100000,0.000000,564.366667,1,31,0,0,0.0,99999825,Gx4sJzcQog01UhZL
1,1,0,10,265.20,261.20,208.50,174.50,0.000000,1,0,...,240.100000,355.166667,0.000000,1,30,0,0,0.0,90063345,kEXrhTiug93DIcLG
2,1,0,12,44.50,70.20,69.00,61.40,2598.397406,0,0,...,27.666667,0.000000,98.416667,2,25,0,0,0.0,90109916,AouXr0EOUtSRdiYK
3,4,0,134,87.95,81.40,76.00,88.30,988.440563,0,0,...,89.900000,74.483333,121.833333,1,44,0,0,0.0,89950166,Yds7U30hnRZDiLtb
4,4,0,84,317.04,314.08,435.51,413.05,5885.800642,0,1,...,0.000000,35.100000,496.733333,1,31,2,1,0.0,99999825,OFDTSXrhN9Q2mbVw


In [10]:
#@func: train for 10 class
def LGB():
        clf = lgb.LGBMClassifier(
                bjective='multiclass',
                boosting_type='gbdt',
                num_leaves=35,
                max_depth=8,
                learning_rate=0.05,
                seed=2018,
                colsample_bytree=0.8,
                subsample=0.9,
                n_estimators=4000)
        return clf

In [6]:
#@func: train for 2 class
def LGB2():
        clf = lgb.LGBMClassifier(
                bjective='binary',
                boosting_type='gbdt',
                num_leaves=1000,
                verbose=0,
                learning_rate=0.05,
                seed=2018,
                subsample=0.9,
                n_estimators=1000,
                feature_fraction=0.9,
                bagging_fraction=0.8,
                bagging_freq=5,
                num_boost_round=2000,
                valid_sets='valid',
                early_stopping_rounds=5 
        )
        return clf

In [7]:
#准备 全模型 训练集
Xtrain = train.drop(['user_id'],axis = 1)
Xtrain_feature = [value for value in Xtrain.columns.values if
                   value not in ['current_service']]
Xtrain[Xtrain_feature].head()

,service_type,is_mix_service,online_time,1_total_fee,2_total_fee,3_total_fee,4_total_fee,month_traffic,many_over_bill,contract_type,...,last_month_traffic,local_trafffic_month,local_caller_time,service1_caller_time,service2_caller_time,gender,age,complaint_level,former_complaint_num,former_complaint_fee
0,4,0,85,295.96,296.20,296.00,296.80,3813.614698,0,1,...,4096.000000,1392.038508,108.100000,0.000000,564.366667,1,31,0,0,0.0
1,1,0,10,265.20,261.20,208.50,174.50,0.000000,1,0,...,0.000000,62852.509718,240.100000,355.166667,0.000000,1,30,0,0,0.0
2,1,0,12,44.50,70.20,69.00,61.40,2598.397406,0,0,...,0.000000,1935.242104,27.666667,0.000000,98.416667,2,25,0,0,0.0
3,4,0,134,87.95,81.40,76.00,88.30,988.440563,0,0,...,37.336425,988.561075,89.900000,74.483333,121.833333,1,44,0,0,0.0
4,4,0,84,317.04,314.08,435.51,413.05,5885.800642,0,1,...,3305.741127,0.000000,0.000000,35.100000,496.733333,1,31,2,1,0.0


In [8]:
#全模型标签  (将27类型替换为26，视为一类进行十分类)
Xtrain['current_service'] = Xtrain['current_service'].replace(99999827,99999826)
print(set(Xtrain['current_service']))
label = Xtrain['current_service']
le = LabelEncoder()
label = le.fit_transform(label)
print(set(label))

{90155946, 99999825, 90063345, 99999826, 99999828, 89950166, 89950167, 89950168, 99999830, 90109916}
{0, 1, 2, 3, 4, 5, 6, 7, 8, 9}


In [9]:
#全模型测试集
Xtest = test.drop(['user_id'],axis = 1)
Xtest_feature = [value for value in Xtest.columns.values if
                   value not in ['current_service']]
Xtest[Xtest_feature].head()

,service_type,is_mix_service,online_time,1_total_fee,2_total_fee,3_total_fee,4_total_fee,month_traffic,many_over_bill,contract_type,...,last_month_traffic,local_trafffic_month,local_caller_time,service1_caller_time,service2_caller_time,gender,age,complaint_level,former_complaint_num,former_complaint_fee
543990,1,0,15,49.1,50.9,44.50,57.60,0.000000,1,0,...,0.000000,0.00000,0.00,0.0,6.666667,1,30,0,0,0.0
543991,4,0,14,62.0,183.3,43.60,67.76,548.944325,0,0,...,0.000000,0.00000,0.00,0.0,51.466667,2,17,0,0,0.0
543992,1,0,6,36.0,36.0,36.00,43.70,2135.174574,0,0,...,943.416368,2216.05873,2.25,0.0,87.233333,1,22,0,0,0.0
543993,1,0,7,78.5,58.0,45.50,51.10,0.000000,1,0,...,0.000000,0.00000,0.00,78.8,4.766667,2,23,0,0,0.0
543994,1,0,4,60.4,64.9,33.29,0.00,0.000000,1,0,...,0.000000,21047.09120,86.80,86.8,0.000000,2,17,0,0,0.0


In [11]:
#全模型训练
model = LGB()
model.fit(Xtrain[Xtrain_feature], label, eval_set=[(Xtrain[Xtrain_feature], label)], verbose=1)
pred = model.predict(Xtest[Xtest_feature])
pred = le.inverse_transform(pred)
test['predict'] = pred

[1]	valid_0's multi_logloss: 1.72966
[2]	valid_0's multi_logloss: 1.61163
[3]	valid_0's multi_logloss: 1.51087
[4]	valid_0's multi_logloss: 1.42461
[5]	valid_0's multi_logloss: 1.34961
[6]	valid_0's multi_logloss: 1.28461
[7]	valid_0's multi_logloss: 1.22552
[8]	valid_0's multi_logloss: 1.16969
[9]	valid_0's multi_logloss: 1.11987
[10]	valid_0's multi_logloss: 1.07349
[11]	valid_0's multi_logloss: 1.03063
[12]	valid_0's multi_logloss: 0.989989
[13]	valid_0's multi_logloss: 0.952356
[14]	valid_0's multi_logloss: 0.917574
[15]	valid_0's multi_logloss: 0.884479
[16]	valid_0's multi_logloss: 0.853762
[17]	valid_0's multi_logloss: 0.824669
[18]	valid_0's multi_logloss: 0.797455
[19]	valid_0's multi_logloss: 0.771926
[20]	valid_0's multi_logloss: 0.74766
[21]	valid_0's multi_logloss: 0.72466
[22]	valid_0's multi_logloss: 0.702959
[23]	valid_0's multi_logloss: 0.682603
[24]	valid_0's multi_logloss: 0.66278
[25]	valid_0's multi_logloss: 0.644433
[26]	valid_0's multi_logloss: 0.626317
[27]	vali

[210]	valid_0's multi_logloss: 0.183749
[211]	valid_0's multi_logloss: 0.183516
[212]	valid_0's multi_logloss: 0.183256
[213]	valid_0's multi_logloss: 0.183017
[214]	valid_0's multi_logloss: 0.182788
[215]	valid_0's multi_logloss: 0.182555
[216]	valid_0's multi_logloss: 0.182344
[217]	valid_0's multi_logloss: 0.182124
[218]	valid_0's multi_logloss: 0.181904
[219]	valid_0's multi_logloss: 0.181684
[220]	valid_0's multi_logloss: 0.181474
[221]	valid_0's multi_logloss: 0.181235
[222]	valid_0's multi_logloss: 0.180982
[223]	valid_0's multi_logloss: 0.180706
[224]	valid_0's multi_logloss: 0.18048
[225]	valid_0's multi_logloss: 0.180264
[226]	valid_0's multi_logloss: 0.180052
[227]	valid_0's multi_logloss: 0.179807
[228]	valid_0's multi_logloss: 0.179569
[229]	valid_0's multi_logloss: 0.179364
[230]	valid_0's multi_logloss: 0.179157
[231]	valid_0's multi_logloss: 0.178937
[232]	valid_0's multi_logloss: 0.17873
[233]	valid_0's multi_logloss: 0.178513
[234]	valid_0's multi_logloss: 0.178276
[2

[416]	valid_0's multi_logloss: 0.153001
[417]	valid_0's multi_logloss: 0.152909
[418]	valid_0's multi_logloss: 0.152817
[419]	valid_0's multi_logloss: 0.152716
[420]	valid_0's multi_logloss: 0.152623
[421]	valid_0's multi_logloss: 0.15251
[422]	valid_0's multi_logloss: 0.152412
[423]	valid_0's multi_logloss: 0.152315
[424]	valid_0's multi_logloss: 0.152224
[425]	valid_0's multi_logloss: 0.152138
[426]	valid_0's multi_logloss: 0.152048
[427]	valid_0's multi_logloss: 0.151967
[428]	valid_0's multi_logloss: 0.151883
[429]	valid_0's multi_logloss: 0.151798
[430]	valid_0's multi_logloss: 0.151692
[431]	valid_0's multi_logloss: 0.15161
[432]	valid_0's multi_logloss: 0.151519
[433]	valid_0's multi_logloss: 0.15143
[434]	valid_0's multi_logloss: 0.151345
[435]	valid_0's multi_logloss: 0.151248
[436]	valid_0's multi_logloss: 0.151153
[437]	valid_0's multi_logloss: 0.151062
[438]	valid_0's multi_logloss: 0.150963
[439]	valid_0's multi_logloss: 0.150878
[440]	valid_0's multi_logloss: 0.150787
[44

[622]	valid_0's multi_logloss: 0.136939
[623]	valid_0's multi_logloss: 0.136869
[624]	valid_0's multi_logloss: 0.136794
[625]	valid_0's multi_logloss: 0.136731
[626]	valid_0's multi_logloss: 0.136677
[627]	valid_0's multi_logloss: 0.13661
[628]	valid_0's multi_logloss: 0.136541
[629]	valid_0's multi_logloss: 0.136477
[630]	valid_0's multi_logloss: 0.136407
[631]	valid_0's multi_logloss: 0.136344
[632]	valid_0's multi_logloss: 0.136286
[633]	valid_0's multi_logloss: 0.136219
[634]	valid_0's multi_logloss: 0.136152
[635]	valid_0's multi_logloss: 0.136082
[636]	valid_0's multi_logloss: 0.136018
[637]	valid_0's multi_logloss: 0.135959
[638]	valid_0's multi_logloss: 0.135898
[639]	valid_0's multi_logloss: 0.135843
[640]	valid_0's multi_logloss: 0.135781
[641]	valid_0's multi_logloss: 0.135728
[642]	valid_0's multi_logloss: 0.135663
[643]	valid_0's multi_logloss: 0.135589
[644]	valid_0's multi_logloss: 0.13552
[645]	valid_0's multi_logloss: 0.135452
[646]	valid_0's multi_logloss: 0.135389
[6

[828]	valid_0's multi_logloss: 0.125021
[829]	valid_0's multi_logloss: 0.124969
[830]	valid_0's multi_logloss: 0.124913
[831]	valid_0's multi_logloss: 0.124859
[832]	valid_0's multi_logloss: 0.124814
[833]	valid_0's multi_logloss: 0.124767
[834]	valid_0's multi_logloss: 0.124715
[835]	valid_0's multi_logloss: 0.124661
[836]	valid_0's multi_logloss: 0.124606
[837]	valid_0's multi_logloss: 0.124551
[838]	valid_0's multi_logloss: 0.124503
[839]	valid_0's multi_logloss: 0.124448
[840]	valid_0's multi_logloss: 0.124396
[841]	valid_0's multi_logloss: 0.124346
[842]	valid_0's multi_logloss: 0.124295
[843]	valid_0's multi_logloss: 0.124244
[844]	valid_0's multi_logloss: 0.124189
[845]	valid_0's multi_logloss: 0.124136
[846]	valid_0's multi_logloss: 0.124089
[847]	valid_0's multi_logloss: 0.124038
[848]	valid_0's multi_logloss: 0.123989
[849]	valid_0's multi_logloss: 0.123939
[850]	valid_0's multi_logloss: 0.123887
[851]	valid_0's multi_logloss: 0.123848
[852]	valid_0's multi_logloss: 0.1238
[8

...

[3395]	valid_0's multi_logloss: 0.0568235
[3396]	valid_0's multi_logloss: 0.0568084
[3397]	valid_0's multi_logloss: 0.0567931
[3398]	valid_0's multi_logloss: 0.0567746
[3399]	valid_0's multi_logloss: 0.0567551
[3400]	valid_0's multi_logloss: 0.0567397
[3401]	valid_0's multi_logloss: 0.0567221
[3402]	valid_0's multi_logloss: 0.0567053
[3403]	valid_0's multi_logloss: 0.0566935
[3404]	valid_0's multi_logloss: 0.0566756
[3405]	valid_0's multi_logloss: 0.0566615
[3406]	valid_0's multi_logloss: 0.0566527
[3407]	valid_0's multi_logloss: 0.0566359
[3408]	valid_0's multi_logloss: 0.0566193
[3409]	valid_0's multi_logloss: 0.0566062
[3410]	valid_0's multi_logloss: 0.0565885
[3411]	valid_0's multi_logloss: 0.056572
[3412]	valid_0's multi_logloss: 0.0565549
[3413]	valid_0's multi_logloss: 0.0565371
[3414]	valid_0's multi_logloss: 0.0565223
[3415]	valid_0's multi_logloss: 0.056505
[3416]	valid_0's multi_logloss: 0.0564877
[3417]	valid_0's multi_logloss: 0.056474
[3418]	valid_0's multi_logloss: 0.056

[3591]	valid_0's multi_logloss: 0.0537676
[3592]	valid_0's multi_logloss: 0.0537525
[3593]	valid_0's multi_logloss: 0.0537384
[3594]	valid_0's multi_logloss: 0.0537273
[3595]	valid_0's multi_logloss: 0.0537148
[3596]	valid_0's multi_logloss: 0.0537018
[3597]	valid_0's multi_logloss: 0.0536859
[3598]	valid_0's multi_logloss: 0.0536727
[3599]	valid_0's multi_logloss: 0.0536614
[3600]	valid_0's multi_logloss: 0.0536506
[3601]	valid_0's multi_logloss: 0.0536315
[3602]	valid_0's multi_logloss: 0.0536168
[3603]	valid_0's multi_logloss: 0.0535993
[3604]	valid_0's multi_logloss: 0.0535857
[3605]	valid_0's multi_logloss: 0.0535707
[3606]	valid_0's multi_logloss: 0.0535548
[3607]	valid_0's multi_logloss: 0.0535412
[3608]	valid_0's multi_logloss: 0.0535251
[3609]	valid_0's multi_logloss: 0.0535078
[3610]	valid_0's multi_logloss: 0.0534927
[3611]	valid_0's multi_logloss: 0.0534795
[3612]	valid_0's multi_logloss: 0.0534655
[3613]	valid_0's multi_logloss: 0.0534523
[3614]	valid_0's multi_logloss: 0.

[3787]	valid_0's multi_logloss: 0.0510609
[3788]	valid_0's multi_logloss: 0.0510465
[3789]	valid_0's multi_logloss: 0.0510322
[3790]	valid_0's multi_logloss: 0.0510186
[3791]	valid_0's multi_logloss: 0.0510057
[3792]	valid_0's multi_logloss: 0.0509915
[3793]	valid_0's multi_logloss: 0.0509803
[3794]	valid_0's multi_logloss: 0.0509678
[3795]	valid_0's multi_logloss: 0.0509546
[3796]	valid_0's multi_logloss: 0.0509401
[3797]	valid_0's multi_logloss: 0.050926
[3798]	valid_0's multi_logloss: 0.0509104
[3799]	valid_0's multi_logloss: 0.0508956
[3800]	valid_0's multi_logloss: 0.0508809
[3801]	valid_0's multi_logloss: 0.0508671
[3802]	valid_0's multi_logloss: 0.0508472
[3803]	valid_0's multi_logloss: 0.05083
[3804]	valid_0's multi_logloss: 0.0508162
[3805]	valid_0's multi_logloss: 0.0508039
[3806]	valid_0's multi_logloss: 0.050787
[3807]	valid_0's multi_logloss: 0.0507725
[3808]	valid_0's multi_logloss: 0.0507586
[3809]	valid_0's multi_logloss: 0.0507447
[3810]	valid_0's multi_logloss: 0.0507

[3983]	valid_0's multi_logloss: 0.0484949
[3984]	valid_0's multi_logloss: 0.04848
[3985]	valid_0's multi_logloss: 0.0484698
[3986]	valid_0's multi_logloss: 0.0484556
[3987]	valid_0's multi_logloss: 0.0484389
[3988]	valid_0's multi_logloss: 0.0484256
[3989]	valid_0's multi_logloss: 0.048415
[3990]	valid_0's multi_logloss: 0.0484012
[3991]	valid_0's multi_logloss: 0.0483872
[3992]	valid_0's multi_logloss: 0.0483745
[3993]	valid_0's multi_logloss: 0.0483644
[3994]	valid_0's multi_logloss: 0.0483542
[3995]	valid_0's multi_logloss: 0.0483428
[3996]	valid_0's multi_logloss: 0.0483309
[3997]	valid_0's multi_logloss: 0.0483223
[3998]	valid_0's multi_logloss: 0.0483137
[3999]	valid_0's multi_logloss: 0.0483003
[4000]	valid_0's multi_logloss: 0.0482845


D:\anocanda\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
D:\anocanda\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


In [13]:
#全模型分类结果
print(set(test['predict']))
test.head()

{90155946, 90063345, 99999826, 99999825, 99999828, 89950167, 89950166, 99999830, 89950168, 90109916}


,service_type,is_mix_service,online_time,1_total_fee,2_total_fee,3_total_fee,4_total_fee,month_traffic,many_over_bill,contract_type,...,service1_caller_time,service2_caller_time,gender,age,complaint_level,former_complaint_num,former_complaint_fee,current_service,user_id,predict
543990,1,0,15,49.1,50.9,44.50,57.60,0.000000,1,0,...,0.0,6.666667,1,30,0,0,0.0,90063345,6Iky7Crmi9U2oeYc,90063345
543991,4,0,14,62.0,183.3,43.60,67.76,548.944325,0,0,...,0.0,51.466667,2,17,0,0,0.0,89950168,6il0f5jstE3DIUAb,89950168
543992,1,0,6,36.0,36.0,36.00,43.70,2135.174574,0,0,...,0.0,87.233333,1,22,0,0,0.0,90109916,6iL7nFvTSUDXpsfw,90109916
543993,1,0,7,78.5,58.0,45.50,51.10,0.000000,1,0,...,78.8,4.766667,2,23,0,0,0.0,90063345,6iL9jYKA0gBNPDWd,90063345
543994,1,0,4,60.4,64.9,33.29,0.00,0.000000,1,0,...,86.8,0.000000,2,17,0,0,0.0,90063345,6ilDfetAok14JhZ0,90063345


In [14]:
#准备 单模型 训练集
Xtrain_single = pd.concat([train[train.current_service == 99999826],train[train.current_service == 99999827]],axis = 0)
Xtrain_single_feature = [value for value in Xtrain_single.columns.values if
                   value not in ['user_id','current_service']]
Xtrain_single[Xtrain_single_feature].head()

,service_type,is_mix_service,online_time,1_total_fee,2_total_fee,3_total_fee,4_total_fee,month_traffic,many_over_bill,contract_type,...,last_month_traffic,local_trafffic_month,local_caller_time,service1_caller_time,service2_caller_time,gender,age,complaint_level,former_complaint_num,former_complaint_fee
20,4,0,60,225.95,339.25,296.45,354.35,500.000000,0,1,...,0.0,1430.710204,86.80,0.0,201.616667,1,36,0,0,0.0
27,4,0,18,146.00,146.10,146.20,146.00,6.233837,0,0,...,3072.0,0.000000,0.00,0.0,39.966667,0,0,0,0,0.0
35,4,0,34,206.66,333.02,241.00,316.30,2563.414215,1,1,...,0.0,2563.526344,68.35,0.0,68.666667,1,33,0,0,0.0
58,4,0,57,201.00,201.00,201.00,201.00,2529.868616,0,0,...,3072.0,2455.083696,56.75,0.0,178.766667,1,39,0,0,0.0
68,4,0,114,207.60,329.50,265.10,205.00,4021.913831,1,1,...,0.0,0.000000,0.00,0.0,67.733333,1,31,0,0,0.0


In [15]:
#单模型标签
print(set(Xtrain_single['current_service']))
label_single = Xtrain_single['current_service']
le_single = LabelEncoder()
label_single = le_single.fit_transform(label_single)
print(set(label_single))

{99999826, 99999827}
{0, 1}


In [16]:
#单模型测试集
#从全模型结果选择99999826单独进行二分类还原为99999826，99999827
test_temp = test[test['predict'].isin([99999826])]
Xtest_single = test_temp.drop(['user_id'],axis = 1)
Xtest_single_feature = [value for value in Xtest_single.columns.values if
                   value not in ['predict','current_service']]
Xtest_single[Xtest_single_feature].head()

,service_type,is_mix_service,online_time,1_total_fee,2_total_fee,3_total_fee,4_total_fee,month_traffic,many_over_bill,contract_type,...,last_month_traffic,local_trafffic_month,local_caller_time,service1_caller_time,service2_caller_time,gender,age,complaint_level,former_complaint_num,former_complaint_fee
543996,4,0,32,196.00,256.00,286.00,232.00,1474.221327,0,1,...,0.0,64.400747,0.000000,0.000000,15.466667,1,27,0,0,0.0
543998,4,0,175,291.26,288.91,240.08,269.10,4381.679605,1,9,...,0.0,2548.636515,418.100000,5.833333,696.316667,1,46,0,0,0.0
544000,4,0,38,195.98,202.72,206.00,195.92,0.000000,0,9,...,0.0,1567.029274,142.116667,0.000000,210.483333,1,27,0,0,0.0
544001,4,0,97,190.20,267.36,190.10,190.00,871.807962,0,9,...,2048.0,513.825382,15.533333,0.000000,19.400000,1,30,0,0,0.0
544054,4,0,16,123.00,141.10,133.00,123.10,5495.797660,0,12,...,0.0,10726.896240,113.133333,0.000000,121.966667,1,19,0,0,0.0


In [17]:
#单模型训练
model_single = LGB2()
model_single.fit(Xtrain_single[Xtrain_single_feature], label_single, eval_set=[(Xtrain_single[Xtrain_single_feature], label_single)], verbose=1)
pred_single = model_single.predict(Xtest_single[Xtest_single_feature])
pred_single = le_single.inverse_transform(pred_single)
test_temp['predict_gbm'] = pred_single

D:\anocanda\lib\site-packages\lightgbm\engine.py:118: UserWarning: Found `num_boost_round` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))
D:\anocanda\lib\site-packages\lightgbm\engine.py:123: UserWarning: Found `early_stopping_rounds` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))


[1]	valid_0's binary_logloss: 0.658444
Training until validation scores don't improve for 5 rounds.
[2]	valid_0's binary_logloss: 0.627405
[3]	valid_0's binary_logloss: 0.598613
[4]	valid_0's binary_logloss: 0.574462
[5]	valid_0's binary_logloss: 0.549902
[6]	valid_0's binary_logloss: 0.527245
[7]	valid_0's binary_logloss: 0.507627
[8]	valid_0's binary_logloss: 0.488351
[9]	valid_0's binary_logloss: 0.469871
[10]	valid_0's binary_logloss: 0.452614
[11]	valid_0's binary_logloss: 0.436651
[12]	valid_0's binary_logloss: 0.421384
[13]	valid_0's binary_logloss: 0.407014
[14]	valid_0's binary_logloss: 0.393438
[15]	valid_0's binary_logloss: 0.38105
[16]	valid_0's binary_logloss: 0.368982
[17]	valid_0's binary_logloss: 0.358226
[18]	valid_0's binary_logloss: 0.347567
[19]	valid_0's binary_logloss: 0.337247
[20]	valid_0's binary_logloss: 0.3275
[21]	valid_0's binary_logloss: 0.318023
[22]	valid_0's binary_logloss: 0.309037
[23]	valid_0's binary_logloss: 0.300468
[24]	valid_0's binary_logloss: 

[200]	valid_0's binary_logloss: 0.0140787
[201]	valid_0's binary_logloss: 0.0138437
[202]	valid_0's binary_logloss: 0.0136236
[203]	valid_0's binary_logloss: 0.0134223
[204]	valid_0's binary_logloss: 0.0132267
[205]	valid_0's binary_logloss: 0.0130316
[206]	valid_0's binary_logloss: 0.0128462
[207]	valid_0's binary_logloss: 0.0126688
[208]	valid_0's binary_logloss: 0.0124888
[209]	valid_0's binary_logloss: 0.0123064
[210]	valid_0's binary_logloss: 0.0121231
[211]	valid_0's binary_logloss: 0.0119283
[212]	valid_0's binary_logloss: 0.0117533
[213]	valid_0's binary_logloss: 0.0115711
[214]	valid_0's binary_logloss: 0.0114001
[215]	valid_0's binary_logloss: 0.011234
[216]	valid_0's binary_logloss: 0.011063
[217]	valid_0's binary_logloss: 0.010883
[218]	valid_0's binary_logloss: 0.0107104
[219]	valid_0's binary_logloss: 0.0105501
[220]	valid_0's binary_logloss: 0.0103946
[221]	valid_0's binary_logloss: 0.0102382
[222]	valid_0's binary_logloss: 0.0100766
[223]	valid_0's binary_logloss: 0.009

[393]	valid_0's binary_logloss: 0.000898214
[394]	valid_0's binary_logloss: 0.000886764
[395]	valid_0's binary_logloss: 0.000875421
[396]	valid_0's binary_logloss: 0.000862078
[397]	valid_0's binary_logloss: 0.000850214
[398]	valid_0's binary_logloss: 0.000837902
[399]	valid_0's binary_logloss: 0.000827038
[400]	valid_0's binary_logloss: 0.000815744
[401]	valid_0's binary_logloss: 0.00080515
[402]	valid_0's binary_logloss: 0.000795734
[403]	valid_0's binary_logloss: 0.000787305
[404]	valid_0's binary_logloss: 0.000777252
[405]	valid_0's binary_logloss: 0.00076849
[406]	valid_0's binary_logloss: 0.000759312
[407]	valid_0's binary_logloss: 0.000750047
[408]	valid_0's binary_logloss: 0.000741384
[409]	valid_0's binary_logloss: 0.000732744
[410]	valid_0's binary_logloss: 0.00072363
[411]	valid_0's binary_logloss: 0.000712585
[412]	valid_0's binary_logloss: 0.000703351
[413]	valid_0's binary_logloss: 0.000695569
[414]	valid_0's binary_logloss: 0.000687168
[415]	valid_0's binary_logloss: 0.0

[580]	valid_0's binary_logloss: 0.000109329
[581]	valid_0's binary_logloss: 0.000108514
[582]	valid_0's binary_logloss: 0.00010813
[583]	valid_0's binary_logloss: 0.000107866
[584]	valid_0's binary_logloss: 0.000106562
[585]	valid_0's binary_logloss: 0.000106544
[586]	valid_0's binary_logloss: 0.000105053
[587]	valid_0's binary_logloss: 0.000103649
[588]	valid_0's binary_logloss: 0.000102425
[589]	valid_0's binary_logloss: 0.000101746
[590]	valid_0's binary_logloss: 0.000100945
[591]	valid_0's binary_logloss: 0.000100008
[592]	valid_0's binary_logloss: 9.86715e-05
[593]	valid_0's binary_logloss: 9.84476e-05
[594]	valid_0's binary_logloss: 9.72253e-05
[595]	valid_0's binary_logloss: 9.66091e-05
[596]	valid_0's binary_logloss: 9.61269e-05
[597]	valid_0's binary_logloss: 9.59782e-05
[598]	valid_0's binary_logloss: 9.46529e-05
[599]	valid_0's binary_logloss: 9.48908e-05
[600]	valid_0's binary_logloss: 9.43037e-05
[601]	valid_0's binary_logloss: 9.39665e-05
[602]	valid_0's binary_logloss: 9

D:\anocanda\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
D:\anocanda\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
D:\anocanda\lib\site-packages\ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [18]:
#单模型二分类 逻辑回归
from sklearn.linear_model import LogisticRegression
LR = LogisticRegression()
LR.fit( Xtrain_single[Xtrain_single_feature] , label_single)
pred_LR = LR.predict(Xtest_single[Xtest_single_feature])
pred_LR = le_single.inverse_transform(pred_LR)
test_temp['predict_LR'] = pred_LR

D:\anocanda\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
D:\anocanda\lib\site-packages\ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  import sys


In [26]:
#单模型再分类结果
test_temp.head()

,service_type,is_mix_service,online_time,1_total_fee,2_total_fee,3_total_fee,4_total_fee,month_traffic,many_over_bill,contract_type,...,gender,age,complaint_level,former_complaint_num,former_complaint_fee,current_service,user_id,predict,predict_gbm,predict_LR
543996,4,0,32,196.00,256.00,286.00,232.00,1474.221327,0,1,...,1,27,0,0,0.0,99999826,6IlHVr8JimNXgGTk,99999826,99999826,99999826
543998,4,0,175,291.26,288.91,240.08,269.10,4381.679605,1,9,...,1,46,0,0,0.0,99999826,6IlmzTeEBJa1XyYD,99999826,99999827,99999827
544000,4,0,38,195.98,202.72,206.00,195.92,0.000000,0,9,...,1,27,0,0,0.0,99999826,6im5h7r01ds9LCqJ,99999826,99999826,99999827
544001,4,0,97,190.20,267.36,190.10,190.00,871.807962,0,9,...,1,30,0,0,0.0,99999827,6Imd3OlKj7tgupGZ,99999826,99999827,99999826
544054,4,0,16,123.00,141.10,133.00,123.10,5495.797660,0,12,...,1,19,0,0,0.0,99999826,6IxWCjNXYBa2tlDu,99999826,99999826,99999827


In [20]:
#全模型/单模型分类结果
test_save = test.copy()
test_other = test_save[~test_save['predict'].isin([99999826])]
test_other.head()

,service_type,is_mix_service,online_time,1_total_fee,2_total_fee,3_total_fee,4_total_fee,month_traffic,many_over_bill,contract_type,...,service1_caller_time,service2_caller_time,gender,age,complaint_level,former_complaint_num,former_complaint_fee,current_service,user_id,predict
543990,1,0,15,49.1,50.9,44.50,57.60,0.000000,1,0,...,0.0,6.666667,1,30,0,0,0.0,90063345,6Iky7Crmi9U2oeYc,90063345
543991,4,0,14,62.0,183.3,43.60,67.76,548.944325,0,0,...,0.0,51.466667,2,17,0,0,0.0,89950168,6il0f5jstE3DIUAb,89950168
543992,1,0,6,36.0,36.0,36.00,43.70,2135.174574,0,0,...,0.0,87.233333,1,22,0,0,0.0,90109916,6iL7nFvTSUDXpsfw,90109916
543993,1,0,7,78.5,58.0,45.50,51.10,0.000000,1,0,...,78.8,4.766667,2,23,0,0,0.0,90063345,6iL9jYKA0gBNPDWd,90063345
543994,1,0,4,60.4,64.9,33.29,0.00,0.000000,1,0,...,86.8,0.000000,2,17,0,0,0.0,90063345,6ilDfetAok14JhZ0,90063345


In [34]:
#组合结果 gbm+gbm
test_1 = test_temp.copy()
test_gbm = test_1.drop(['predict_LR'],axis=1)
test_gbm = test_gbm.drop(['predict'],axis=1)
test_gbm.rename(columns={'predict_gbm':'predict'}, inplace = True)
test_new_gbm = pd.concat([test_other,test_gbm],axis = 0)
test_new_gbm.head()

,service_type,is_mix_service,online_time,1_total_fee,2_total_fee,3_total_fee,4_total_fee,month_traffic,many_over_bill,contract_type,...,service1_caller_time,service2_caller_time,gender,age,complaint_level,former_complaint_num,former_complaint_fee,current_service,user_id,predict
543990,1,0,15,49.1,50.9,44.50,57.60,0.000000,1,0,...,0.0,6.666667,1,30,0,0,0.0,90063345,6Iky7Crmi9U2oeYc,90063345
543991,4,0,14,62.0,183.3,43.60,67.76,548.944325,0,0,...,0.0,51.466667,2,17,0,0,0.0,89950168,6il0f5jstE3DIUAb,89950168
543992,1,0,6,36.0,36.0,36.00,43.70,2135.174574,0,0,...,0.0,87.233333,1,22,0,0,0.0,90109916,6iL7nFvTSUDXpsfw,90109916
543993,1,0,7,78.5,58.0,45.50,51.10,0.000000,1,0,...,78.8,4.766667,2,23,0,0,0.0,90063345,6iL9jYKA0gBNPDWd,90063345
543994,1,0,4,60.4,64.9,33.29,0.00,0.000000,1,0,...,86.8,0.000000,2,17,0,0,0.0,90063345,6ilDfetAok14JhZ0,90063345


In [38]:
test_2 = test_temp.copy()
test_lr = test_2.drop(['predict_gbm'],axis=1)
test_new_lr = pd.concat([test_other,test_lr],axis = 0)
#组合结果 gbm+logistc regression
test_2 = test_temp.copy()
test_lr = test_2.drop(['predict_gbm'],axis=1)
test_lr = test_lr.drop(['predict'],axis=1)
test_lr.rename(columns={'predict_LR':'predict'}, inplace = True)
test_new_lr = pd.concat([test_other,test_lr],axis = 0)
test_new_lr.head()

D:\anocanda\lib\site-packages\ipykernel_launcher.py:3: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  This is separate from the ipykernel package so we can avoid doing imports until


,service_type,is_mix_service,online_time,1_total_fee,2_total_fee,3_total_fee,4_total_fee,month_traffic,many_over_bill,contract_type,...,service1_caller_time,service2_caller_time,gender,age,complaint_level,former_complaint_num,former_complaint_fee,current_service,user_id,predict
543990,1,0,15,49.1,50.9,44.50,57.60,0.000000,1,0,...,0.0,6.666667,1,30,0,0,0.0,90063345,6Iky7Crmi9U2oeYc,90063345
543991,4,0,14,62.0,183.3,43.60,67.76,548.944325,0,0,...,0.0,51.466667,2,17,0,0,0.0,89950168,6il0f5jstE3DIUAb,89950168
543992,1,0,6,36.0,36.0,36.00,43.70,2135.174574,0,0,...,0.0,87.233333,1,22,0,0,0.0,90109916,6iL7nFvTSUDXpsfw,90109916
543993,1,0,7,78.5,58.0,45.50,51.10,0.000000,1,0,...,78.8,4.766667,2,23,0,0,0.0,90063345,6iL9jYKA0gBNPDWd,90063345
543994,1,0,4,60.4,64.9,33.29,0.00,0.000000,1,0,...,86.8,0.000000,2,17,0,0,0.0,90063345,6ilDfetAok14JhZ0,90063345


In [40]:
#F1与准确率
y_true = test_new_gbm['current_service'].tolist()
y_pred = test_new_gbm['predict'].tolist()
print("10+2 LightGBM F1 Score:",f1_score(y_true, y_pred, average='macro'))
acc = 0
for i in range(len(y_pred)):
    if(y_pred[i] ==y_true[i]):
        acc+=1
print("10+2 LightGBM Accuracy Score:",acc/len(y_pred))
#F1与准确率
y_true = test_new_lr['current_service'].tolist()
y_pred = test_new_lr['predict'].tolist()
print("10+2 LightGBM+Logistic Regression F1 Score:",f1_score(y_true, y_pred, average='macro'))
acc = 0
for i in range(len(y_pred)):
    if(y_pred[i] ==y_true[i]):
        acc+=1
print("10+2 LightGBM+Logistic Regression Accuracy Score:",acc/len(y_pred))

10+2 LightGBM F1 Score: 0.9122554805839841
10+2 LightGBM Accuracy Score: 0.9442544425444255
10+2 LightGBM+Logistic Regression F1 Score: 0.8705757563618172
10+2 LightGBM+Logistic Regression Accuracy Score: 0.9248742487424875


In [ ]:
#观察特征

In [98]:
service = set(test['current_service'])
print(service)

{90155946, 90063345, 99999826, 99999827, 99999828, 99999825, 89950166, 99999830, 89950168, 89950167, 90109916}


In [108]:
test_service = test['current_service'].groupby(test['predict'])
print(test_service.count())

predict
89950166    38193
89950167    18568
89950168     8307
90063345    77071
90109916    10249
90155946     5854
99999825     5882
99999826     8673
99999827     8557
99999828    13408
99999830     5236
Name: current_service, dtype: int64


In [125]:
df = pd.DataFrame()
df = test_service.count().reset_index()
df

,predict,current_service
0,89950166,38193
1,89950167,18568
2,89950168,8307
3,90063345,77071
4,90109916,10249
5,90155946,5854
6,99999825,5882
7,99999826,8673
8,99999827,8557
9,99999828,13408


In [136]:
ab = df['current_service']
ab.tolist()

[38193, 18568, 8307, 77071, 10249, 5854, 5882, 8673, 8557, 13408, 5236]

In [131]:
cur_ser = test['current_service'].tolist()
pre = test['predict'].tolist()

In [135]:
cntall = []
for i in ab:
    cnt = 0
    for j in range(len(cur_ser)):
        if cur_ser[j] == i and pre[j] == i:
            cnt += 1
    cntall.append(cnt)
cntall        

[33040, 15186, 7885, 76711, 10166, 5530, 5125, 5939, 5563, 9924, 4039]

In [137]:
accuracy = []
for i in range(len(ab)):
    accuracy.append(cntall[i]/ab[i])
accuracy

[0.8650799884795644,
 0.8178586816027574,
 0.9491994703262309,
 0.9953289823669085,
 0.9919016489413601,
 0.9446532285616672,
 0.8713022781366881,
 0.6847688227833506,
 0.6501110202173659,
 0.7401551312649165,
 0.7713903743315508]